In [17]:
import pandas as pd
import numpy as np
from functools import reduce
from datetime import datetime
from collections import Counter
import itertools
import math
import operator
import json

In [18]:
# Config

BENCHMARK = "TPCH"
CHUNK_SIZE = 65535

if BENCHMARK == "TPCH":
    SCALE_FACTOR = 1
    RUNS = 10
    TIME = 60
    STATISTICS_PATH = f"~/Dokumente/repos/example_plugin/TPC-H__SF_{SCALE_FACTOR}.000000__RUNS_{RUNS}__TIME_{TIME}"
elif BENCHMARK == "TPCDS":
    SCALE_FACTOR = 1
    RUNS = 1
    TIME = 60
    STATISTICS_PATH = f"~/Dokumente/repos/example_plugin/TPC-DS__SF_{SCALE_FACTOR}.000000__RUNS_{RUNS}__TIME_{TIME}"
else:
    raise Exception("Unknown benchmark: " + BENCHMARK)

print(f"Model is configured for {BENCHMARK} (chunk size {CHUNK_SIZE}) with scale factor {SCALE_FACTOR}, {TIME} seconds runtime, and at most {RUNS} runs per query")

Model is configured for TPCH (chunk size 65535) with scale factor 1, 60 seconds runtime, and at most 10 runs per query


In [19]:
# Load table scan statistics

path = f"{STATISTICS_PATH}/table_scans.csv"
scans = pd.read_csv(path, sep='|')
EXPECTED_SCAN_COUNT = len(scans)
LOADED_CHUNK_SIZE = CHUNK_SIZE
LOADED_BENCHMARK = BENCHMARK
LOADED_SCALE_FACTOR = SCALE_FACTOR
LOADED_RUNS = RUNS
LOADED_TIME = TIME

print(f"Successfully loaded {path}")

def assert_correct_statistics_loaded():
    assert BENCHMARK == LOADED_BENCHMARK, f"The model is configured to use {BENCHMARK}, but {LOADED_BENCHMARK} is currently loaded.\nEither change the benchmark or re-run all cells"
    assert SCALE_FACTOR == LOADED_SCALE_FACTOR, f"The model is configured to use {SCALE_FACTOR} as scale factor, but data for a scale factor of {LOADED_SCALE_FACTOR} is currently loaded.\nEither change the benchmark or re-run all cells"
    assert RUNS == LOADED_RUNS, f"The model is configured to perform at most {RUNS} runs, but the currently loaded data had at most {LOADED_RUNS} runs.\nEither change the benchmark or re-run all cells"
    assert TIME == LOADED_TIME, f"The model is configured to run for {TIME} seconds, but the currently data had a runtime of {LOADED_TIME} seconds.\nEither change the benchmark or re-run all cells"
    assert CHUNK_SIZE == LOADED_CHUNK_SIZE, f"The model is configured to use {CHUNK_SIZE} as chunk_size, but data for a chunk size of {LOADED_CHUNK_SIZE} is currently loaded.\nEither change the benchmark or re-run all cells"
    assert EXPECTED_SCAN_COUNT == len(scans), f"There should be {EXPECTED_SCAN_COUNT} table scans, but there are only {len(scans)}\nProbably one of the last commands reassigned it unintentionally"
    
    assert 'GET_TABLE_HASH' in scans.columns, f"the statistics in {STATISTICS_PATH} are outdated (column 'GET_TABLE_HASH' in table_scans.csv is missing). Please create them again."

Successfully loaded ~/Dokumente/repos/example_plugin/TPC-H__SF_1.000000__RUNS_10__TIME_60/table_scans.csv


In [20]:
# Validate table scans
assert_correct_statistics_loaded()

# To make sure pruning was not active,
# first fetch table sizes,
table_statistics = pd.read_csv(f"{STATISTICS_PATH}/table_meta_data.csv", sep='|')
table_sizes = dict(zip(table_statistics.TABLE_NAME, table_statistics.ROW_COUNT))

# then make sure INPUT_ROWS == table_size
def input_size_matches(row):
    #print(row)
    
    actual_row_count = row['INPUT_ROWS']
    table = row['TABLE_NAME']
    expected_row_count = table_sizes[table]
    return expected_row_count == actual_row_count

data_scans = scans[scans['COLUMN_TYPE'] == 'DATA']
input_size_matches = data_scans.apply(input_size_matches, axis=1)
all_sizes_match = reduce(np.logical_and, input_size_matches) #input_size_matches.apply()

if not all_sizes_match:
    raise Exception("The given statistics were probably created while pruning was active")
else:
    print("OK - looks like pruning was deactivated while the statistics were created")

OK - looks like pruning was deactivated while the statistics were created


In [21]:
# Append additional information to the table scans
assert_correct_statistics_loaded()

print(f"Statistics for {BENCHMARK} contain {len(scans)} table scans")


# Add statistics about selectivity and speed for each operator
scans['selectivity'] = scans['OUTPUT_ROWS'] / scans['INPUT_ROWS']

# TODO: Assumption that reading and writing a row have the same cost
scans['time_per_row'] = scans['RUNTIME_NS'] / (scans['INPUT_ROWS'] + scans['OUTPUT_ROWS'])
scans['time_per_input_row'] = scans['time_per_row']
scans['time_per_output_row'] = scans['time_per_row']


def determine_or_chains(table_scans):
    table_scans['part_of_or_chain'] = False
    
    single_table_scans = table_scans.groupby(['QUERY_HASH', 'TABLE_NAME', 'GET_TABLE_HASH'])
    
    for _, scans in single_table_scans:            
        input_row_frequencies = Counter(scans.INPUT_ROWS)
        or_input_sizes = set([input_size for input_size, frequency in input_row_frequencies.items() if frequency > 1])

        df = pd.DataFrame()
        df['INPUT_ROWS'] = scans['INPUT_ROWS']
        df['OUTPUT_ROWS'] = scans['OUTPUT_ROWS']
        df['part_of_or_chain'] = scans.apply(lambda row: row['INPUT_ROWS'] in or_input_sizes, axis=1)

        for _ in range(len(scans)):
            or_input_sizes |= set(df[df['part_of_or_chain']].OUTPUT_ROWS.unique())
            df['part_of_or_chain'] = df.apply(lambda row: row['INPUT_ROWS'] in or_input_sizes, axis=1)

        or_chains = list(df[df['part_of_or_chain']].index)
        table_scans.iloc[or_chains, table_scans.columns.get_loc('part_of_or_chain')] = True
    
    return table_scans

# Hyrise does not use scans that are part of an OR-chain for pruning
scans = determine_or_chains(scans)


# Like scans are not useful if they start with %
# TODO what if they dont start with % and contain more than one % ? -> up to first % prunable, but is it used?
def benefits_from_sorting(row):    
    description = row['DESCRIPTION']
    if "ColumnLike" in description:
        words = description.split()
        like_criteria = words[-1]
        assert "%" in like_criteria, f"LIKE operators should have an %, but found none in {like_criteria}"
        return like_criteria[1] != '%'
    elif "ExpressionEvaluator" in description and " IN " in description:
        return False
    else:
        return True

scans['benefits_from_sorting'] = scans.apply(benefits_from_sorting, axis=1)
# TODO: valid atm, but feels a bit hacky to assume not benefitting from sorted segments -> not benefitting from pruning
scans['useful_for_pruning'] = scans.apply(lambda row: not row['part_of_or_chain'] and row['benefits_from_sorting'] , axis=1)
EXPECTED_SCAN_COUNT = len(scans)
print(f"Of those, only {len(scans[scans['useful_for_pruning']])} are useful for pruning")

print("TODO: For now, filtering on scans is deactivated. This is because all scans are needed to recognize OR-Chains. Models have to take care themselves whether a scan can contribute to pruning or not")

Statistics for TPCH contain 436 table scans
Of those, only 228 are useful for pruning
TODO: For now, filtering on scans is deactivated. This is because all scans are needed to recognize OR-Chains. Models have to take care themselves whether a scan can contribute to pruning or not


In [22]:
def test_determine_or_chains():
    test = pd.DataFrame()
    test['QUERY_HASH'] = pd.Series(['1']*3  + ['2']*4)
    test['TABLE_NAME'] = pd.Series(['lineitem']*3  + ['part']*4)
    test['GET_TABLE_HASH'] = pd.Series(['0x1'] + ['0x2']*2 + ['0x3']*4)
    test['COLUMN_NAME'] = pd.Series(['l_shipdate', 'l_shipdate', 'l_discount', 'p_brand', 'p_type', 'p_type', 'p_size'])
    test['INPUT_ROWS'] = pd.Series( [6001215, 6001215, 200000, 200000, 199000, 199000, 50000])
    test['OUTPUT_ROWS'] = pd.Series([ 400000,  300000, 200000, 199000,      0,  50000, 20000])
    test_result = determine_or_chains(test)
    assert len(test_result) == 7, "should not filter out any rows"    
    assert len(test_result[test_result['part_of_or_chain']]) == 3, "expected 3 scans, got\n" + str(test_result)
    assert list(test_result['part_of_or_chain']) == [False]*4 + [True]*3
    print("Test OK")

test_determine_or_chains()

Test OK


In [23]:
# Load query frequency information
assert_correct_statistics_loaded()

def get_query_frequencies():
    plan_cache = pd.read_csv(f"{STATISTICS_PATH}/plan_cache.csv", sep='|')
    return dict(zip(plan_cache.QUERY_HASH, plan_cache.EXECUTION_COUNT))

#get_query_frequencies()

In [24]:
# Load column statistics - especially interesting: number of distinct values, and columns sorted during statistics creation

# Returns a 2-level-dictionary: distinct_values[TABLE][COLUMN] = number_of_distinct_values
def get_distinct_values_count():        
    # Code
    column_statistics_df = pd.read_csv(f"{STATISTICS_PATH}/column_meta_data.csv", sep='|')
    column_statistics_df['DISTINCT_VALUES'] = np.int32(column_statistics_df['DISTINCT_VALUES'])
    tables_and_columns = column_statistics_df.groupby('TABLE_NAME')
    distinct_values = {table: dict(zip(column_df.COLUMN_NAME, column_df.DISTINCT_VALUES)) for table, column_df in tables_and_columns }

    
    # Test
    num_tables = len(distinct_values)
    if BENCHMARK == "TPCH":
        assert num_tables == 8, f"TPCH has 8 tables, but got {num_tables}"
    elif BENCHMARK == "TPCDS":
        assert num_tables == 24, f"TPCDS has 24 tables, but got {num_tables}"
    else:
        assert False, "Insert a benchmark specific check here"
    
    return distinct_values

# Returns a dictionary: sorted_columns_during_creation[TABLE] = [column1, column2, ...]
def get_sorted_columns_during_creation():
    # Code
    column_statistics_df = pd.read_csv(f"{STATISTICS_PATH}/column_meta_data.csv", sep='|')
    globally_sorted_columns = column_statistics_df[column_statistics_df['IS_GLOBALLY_SORTED'] == 1]
    
    tables_and_columns = globally_sorted_columns.groupby('TABLE_NAME')
    globally_sorted_columns = {table: list(column_df.COLUMN_NAME) for table, column_df in tables_and_columns }
    
    return globally_sorted_columns

#get_distinct_values_count()
#get_sorted_columns_during_creation()

In [ ]:
### JOINS ###

assert_correct_statistics_loaded()

def load_join_statistics():
    def line_looks_suspicious(row):
        right_table_name = row['RIGHT_TABLE_NAME']    
        if pd.isnull(right_table_name):
            pass
        elif row['RIGHT_TABLE_ROW_COUNT'] > table_sizes[row['RIGHT_TABLE_NAME']]:
            return True

        left_table_name = row['LEFT_TABLE_NAME']
        if pd.isnull(left_table_name):
            pass
        elif row['LEFT_TABLE_ROW_COUNT'] > table_sizes[row['LEFT_TABLE_NAME']]:
            return True

        return False
    
    def validate_joins(joins):
        is_suspicious = joins.apply(line_looks_suspicious, axis=1)
        suspicious_joins = joins[is_suspicious]
        assert len(suspicious_joins) < 3, f"there are {len(suspicious_joins)} suspicious joins:\n{suspicious_joins[['JOIN_MODE', 'LEFT_TABLE_NAME', 'LEFT_COLUMN_NAME', 'LEFT_TABLE_ROW_COUNT', 'RIGHT_TABLE_NAME', 'RIGHT_COLUMN_NAME', 'RIGHT_TABLE_ROW_COUNT', 'OUTPUT_ROWS']]}"
    
    joins = pd.read_csv(f"{STATISTICS_PATH}/joins.csv", sep='|')
    joins['PROBE_TABLE'] = joins.apply(lambda x: x[x['PROBE_SIDE'] + "_TABLE_NAME"], axis=1)
    joins['PROBE_COLUMN'] = joins.apply(lambda x: x[x['PROBE_SIDE'] + "_COLUMN_NAME"], axis=1)
    joins['PROBE_TABLE_ROW_COUNT'] = joins.apply(lambda x: x[x['PROBE_SIDE'] + "_TABLE_ROW_COUNT"], axis=1)
    joins['BUILD_TABLE'] = joins.apply(lambda x: x[x['BUILD_SIDE'] + "_TABLE_NAME"], axis=1)
    joins['BUILD_COLUMN'] = joins.apply(lambda x: x[x['BUILD_SIDE'] + "_COLUMN_NAME"], axis=1)
    joins['BUILD_TABLE_ROW_COUNT'] = joins.apply(lambda x: x[x['BUILD_SIDE'] + "_TABLE_ROW_COUNT"], axis=1)
    validate_joins(joins)
    
    
    # TODOS
    
    # code: export which side is probe
    # model: add columns for probe and build table size
    # model: use more precise values for build and probe materialization
        
    
    
    #TODO code: activate higher cache size    
                                                                                           
    return joins

load_join_statistics()#.iloc[9:10][['JOIN_MODE', 'LEFT_TABLE_NAME', 'LEFT_COLUMN_NAME', 'LEFT_TABLE_ROW_COUNT', 'RIGHT_TABLE_NAME', 'RIGHT_COLUMN_NAME', 'RIGHT_TABLE_ROW_COUNT', 'OUTPUT_ROWS']]

In [26]:
class AbstractModel:
    
    def __init__(self, query_frequencies, table_name, table_scans, correlations={}):
        self.query_frequencies = query_frequencies
        self.table_name = table_name
        self.table_scans = table_scans
        self.correlations = correlations
        
    def query_frequency(self, query_hash):
        return self.query_frequencies[query_hash]
        
    def extract_scan_columns(self):
        useful_scans = self.table_scans[self.table_scans['useful_for_pruning']]
        interesting_scan_columns = list(useful_scans['COLUMN_NAME'].unique())
        
        return interesting_scan_columns
    
    def extract_join_columns(self):
        interesting_join_probe_columns = list(self.joins[self.joins['PROBE_TABLE'] == self.table_name]['PROBE_COLUMN'].unique())
        interesting_join_build_columns = list(self.joins[self.joins['BUILD_TABLE'] == self.table_name]['BUILD_COLUMN'].unique())        
        
        return self.uniquify(interesting_join_probe_columns + interesting_join_build_columns)
    
    def extract_interesting_columns(self):        
        return self.uniquify(self.extract_scan_columns() + self.extract_join_columns())
    
    def round_up_to_next_multiple(self, number_to_round, base_for_multiple):
        quotient = number_to_round // base_for_multiple
        if number_to_round % base_for_multiple != 0:
            quotient += 1
        return quotient * base_for_multiple        

    def uniquify(self, seq):
            seen = set()
            return [x for x in seq if not (x in seen or seen.add(x))]    
    
    # return a list of possible clusterings
    def suggest_clustering(self, first_k=1):
        raise NotImplemented()

In [31]:
class PartitionerModel(AbstractModel):
    
    def __init__(self, query_frequencies, table_name, table_scans, table_size, distinct_values, target_chunksize, correlations, joins, sorted_columns_during_creation):
        super().__init__(query_frequencies, table_name, table_scans, correlations)
        self.table_size = table_size
        self.distinct_values = distinct_values
        self.target_chunksize = target_chunksize
        self.joins = joins
        self.sorted_columns_during_creation = sorted_columns_during_creation
    
    def suggest_clustering(self, first_k=1):
        interesting_columns = self.extract_interesting_columns()

        print(interesting_columns)
        
        clustering_columns = itertools.product(interesting_columns, interesting_columns)
        #clustering_columns = itertools.product(interesting_columns, interesting_columns, interesting_columns)
        clustering_columns = filter(lambda x: x[0] <= x[1], clustering_columns)
        #clustering_columns = filter(lambda x: x[1] <= x[2], clustering_columns)
        clustering_columns = [self.uniquify(clustering) for clustering in clustering_columns]
        sort_columns = interesting_columns        
        clusterings_with_runtimes = reduce(lambda x,y: x+y,[self.estimate_total_runtime(clustering_cols, sort_columns) for clustering_cols in clustering_columns])
        clusterings_with_runtimes.sort(key=lambda x: x[2], reverse=False)
        
        return clusterings_with_runtimes[0:first_k]
        
    def estimate_table_scan_runtimes(self, clustering_columns, sorting_columns, split_factors, total_runtimes):        
        def compute_unprunable_parts(row, split_factors):
            def clustering_columns_correlated_to(column):
                return [clustering_column for clustering_column in clustering_columns if column in self.correlations.get(clustering_column, {})]
            
            def correlates_to_clustering_column(column):
                return len(clustering_columns_correlated_to(column)) > 0

            column_name = row['COLUMN_NAME']

            if not row['useful_for_pruning']:
                selectivity = 1
            elif column_name in clustering_columns:
                scan_selectivity = row['selectivity']
                split_factor = split_factors[clustering_columns.index(column_name)]
                selectivity =  self.round_up_to_next_multiple(scan_selectivity, 1 / split_factor)
            elif correlates_to_clustering_column(column_name):
                scan_selectivity = row['selectivity']
                correlated_clustering_columns = clustering_columns_correlated_to(column_name)
                
                # ToDo this is hacky, but for now assume there is just one correlated column
                assert len(correlated_clustering_columns) == 1, f"expected just 1 correlated clustering column, but got {len(correlated_clustering_columns)}"
                
                split_factor = split_factors[clustering_columns.index(correlated_clustering_columns[0])]
                selectivity = min(1, 1.2 * self.round_up_to_next_multiple(scan_selectivity, 1 / split_factor))
            else:
                selectivity = 1
            
            return selectivity
        
        def compute_runtimes(row, sorting_column):
            assert row['estimated_input_rows'] > 1, row
            assert row['runtime_per_input_row'] > 0, row
            assert row['runtime_per_output_row'] > 0, row
            input_row_count = row['estimated_input_rows']
            
            if row['COLUMN_NAME'] == sorting_column and row['benefits_from_sorting']:
                # TODO is this the best way to simulate sorted access?
                input_row_count = np.log2(input_row_count)

            runtime = input_row_count * row['runtime_per_input_row'] + row['OUTPUT_ROWS'] * row['runtime_per_output_row']
            return runtime * self.query_frequency(row['QUERY_HASH'])
        
        scans_per_query = self.table_scans.sort_values(['INPUT_ROWS'], ascending=False).groupby(['QUERY_HASH', 'GET_TABLE_HASH'])
        for _, scans in scans_per_query:
            number_of_scans = len(scans)
            assert number_of_scans > 0 and number_of_scans < 25, f"weird scan length: {number_of_scans}\nScans:\n{scans}"
            # TODO: kinda unrealistic assumption: everything not in the table scan result can be pruned
                          
            unprunable_parts = scans.apply(compute_unprunable_parts, axis=1, args=(split_factors,))            
            unprunable_part = unprunable_parts.product()
            assert unprunable_part > 0, "no unprunable part"
            
            estimated_pruned_table_size = self.round_up_to_next_multiple(unprunable_part * self.table_size, CHUNK_SIZE)
            
            runtimes = pd.DataFrame()
            runtimes['QUERY_HASH'] = scans['QUERY_HASH']
            runtimes['runtime_per_input_row'] = scans['time_per_input_row']
            runtimes['runtime_per_output_row'] = scans['time_per_output_row']
            runtimes['COLUMN_NAME'] = scans['COLUMN_NAME']
            runtimes['benefits_from_sorting'] = scans['benefits_from_sorting']
            # the pruned table inputs should be reflected in 'estimated_input_rows'
            runtimes['estimated_input_rows'] = scans.apply(lambda x: x['INPUT_ROWS'], axis=1)
            runtimes['OUTPUT_ROWS'] = scans['OUTPUT_ROWS']

            runtimes.iloc[0, runtimes.columns.get_loc('estimated_input_rows')] = estimated_pruned_table_size                                    
            assert runtimes['estimated_input_rows'].iloc[0] == estimated_pruned_table_size, f"value is {runtimes.iloc[0]['estimated_input_rows']}, but should be {estimated_pruned_table_size}"
            # TODO modify input sizes of subsequent scans
            
            for sorting_column in sorting_columns:
                scan_runtimes = runtimes.apply(compute_runtimes, axis=1, args=(sorting_column,))
                total_runtimes[sorting_column] += scan_runtimes.sum()

    def estimate_join_runtimes(self, clustering_columns, sorting_columns, total_runtimes):                
        def estimate_join_runtime(row, sorting_column):
                        
            if "JoinHash" in row['DESCRIPTION']:
                probe_column = row['PROBE_COLUMN']
                if row['PROBE_TABLE'] == self.table_name:
                    probe_column_was_sorted = row['PROBE_SORTED'] and probe_column in self.sorted_columns_during_creation.get(self.table_name, {})
                    probe_column_is_sorted = row['PROBE_SORTED'] and probe_column == sorting_column
                    probe_column_is_clustered = row['PROBE_SORTED'] and probe_column in clustering_columns
                else:
                    probe_column_was_sorted = row['PROBE_SORTED'] and probe_column in self.sorted_columns_during_creation.get(row['PROBE_TABLE'], {})
                    probe_column_is_sorted = probe_column_was_sorted
                    probe_column_is_clustered = probe_column_was_sorted
                    
                build_column = row['BUILD_COLUMN']
                if row['BUILD_TABLE'] == self.table_name:
                    build_column_was_sorted = row['BUILD_SORTED'] and build_column in self.sorted_columns_during_creation.get(self.table_name, {})
                    build_column_is_sorted = row['BUILD_SORTED'] and build_column == sorting_column
                    build_column_is_clustered = row['BUILD_SORTED'] and build_column in clustering_columns
                else:
                    build_column_was_sorted = row['BUILD_SORTED'] and build_column in self.sorted_columns_during_creation.get(row['BUILD_TABLE'], {})
                    build_column_is_sorted = build_column_was_sorted
                    build_column_is_clustered = build_column_was_sorted

                time_materialize = row['BUILD_SIDE_MATERIALIZING_NS'] + row['PROBE_SIDE_MATERIALIZING_NS']
                
                probe_weight = 2
                build_weight = 2
                if probe_column_was_sorted:
                    probe_weight = 1
                if build_column_was_sorted:
                    build_weight = 1
                
                
                probe_table_size = row['PROBE_TABLE_ROW_COUNT']
                build_table_size = row['BUILD_TABLE_ROW_COUNT']
                total_table_size = probe_weight * probe_table_size + build_weight * build_table_size
                
                time_materialize_probe = time_materialize * (probe_weight * probe_table_size / total_table_size)
                time_materialize_build = time_materialize - time_materialize_probe
                
                
                def get_materialize_factor(was_sorted, is_sorted, is_clustered):
                    materialize_factor = 1
                    if is_sorted and is_clustered:
                        if not was_sorted:
                            materialize_factor = 0.5
                        else:
                            materialize_factor = 1
                    elif is_sorted or is_clustered:
                        if not was_sorted:
                            materialize_factor = 0.55
                        else:
                            materialize_factor = 1.1
                    elif was_sorted:
                        # probe column is now neither sorted nor clustered
                        materialize_factor = 2
                    else:
                        # default case: was not sorted before, and is neither sorted nor clustered now. No change
                        materialize_factor = 1
                        
                    return materialize_factor
                
                materialize_probe_factor = get_materialize_factor(probe_column_was_sorted, probe_column_is_sorted, probe_column_is_clustered)
                materialize_build_factor = get_materialize_factor(build_column_was_sorted, build_column_is_sorted, build_column_is_clustered)
                
                time_materialize = time_materialize_probe * materialize_probe_factor + time_materialize_build *  materialize_build_factor
                

                # unchanged
                time_cluster = row['CLUSTERING_NS']
                
                # unchanged
                time_build = row['BUILDING_NS']
                
                            
                time_probe = row['PROBING_NS']
                probe_factor = 1
                if probe_column_is_sorted and probe_column_is_clustered:
                    if not probe_column_was_sorted:
                        probe_factor = 0.7
                    else:
                        probe_factor = 1
                elif probe_column_is_sorted or probe_column_is_clustered:
                    if not probe_column_was_sorted:
                        probe_factor = 0.9
                    else:
                        probe_factor = 1.1
                elif probe_column_was_sorted:
                    # probe column is now neither sorted nor clustered
                    probe_factor = 1.4
                
                time_probe *= probe_factor                
                
                # unchanged
                time_write_output = row['OUTPUT_WRITING_NS']
                
                
                
                # TODO: how to deal with the difference between RUNTIME_NS and sum(stage_runtimes)?
                runtime = time_materialize + time_cluster + time_build + time_probe + time_write_output
            else:
                runtime = row['RUNTIME_NS']
                
            return runtime * self.query_frequency(row['QUERY_HASH'])
        
        for sorting_column in sorting_columns:
            join_runtimes = self.joins.apply(estimate_join_runtime, axis=1, args=(sorting_column,))
            total_runtimes[sorting_column] += join_runtimes.sum()
                
    def estimate_total_runtime(self, clustering_columns, sorting_columns):
        #print(f"testing clustering {clustering_columns} with sorting columns {sorting_columns}")
        split_factors = self.determine_split_factors(clustering_columns)            
        total_runtimes = {sorting_column: 0 for sorting_column in sorting_columns}
        self.estimate_table_scan_runtimes(clustering_columns, sorting_columns, split_factors, total_runtimes)
        self.estimate_join_runtimes(clustering_columns, sorting_columns, total_runtimes)
        
        clusterings = [[list(zip(clustering_columns, split_factors)), sorting_column, np.int64(total_runtimes[sorting_column])] for sorting_column in sorting_columns]
        return clusterings
    
    def determine_split_factors(self, clustering_columns):
        approximate_split_factor = self.table_size / self.target_chunksize
        individual_distinct_values = [self.distinct_values[column] for column in clustering_columns]        
        log_distinct_values = [math.ceil(0.5+np.log2(x)) for x in individual_distinct_values]
        log_distinct_values_product = reduce(operator.mul, log_distinct_values, 1)
        assert log_distinct_values_product > 0, "cannot have a distinct value count of 0"
        
        global_modification_factor = approximate_split_factor / log_distinct_values_product
        num_dimensions = len(clustering_columns)
        individual_modification_factor = np.power(global_modification_factor, 1.0 / num_dimensions)    
        split_factors = [math.ceil(x * individual_modification_factor) for x in log_distinct_values]
        
        # testing
        actual_split_factor = reduce(operator.mul, split_factors, 1)
        assert actual_split_factor > 0, "there was a split up factor of 0"
        estimated_chunksize = self.table_size / actual_split_factor
        assert estimated_chunksize <= self.target_chunksize, "chunks should be smaller, not larger than target_chunksize"
        allowed_percentage = 0.55
        if estimated_chunksize < allowed_percentage * self.target_chunksize:
            print(f"Warning: chunks should not be too much smaller than target_chunksize: {estimated_chunksize} < {allowed_percentage} * {self.target_chunksize}")
        #assert estimated_chunksize >= allowed_percentage * self.target_chunksize, f"chunks should not be too much smaller than target_chunksize: {estimated_chunksize} < {allowed_percentage} * {self.target_chunksize}"
        
        return split_factors    

In [ ]:
assert_correct_statistics_loaded()

def extract_single_table(table_scans, table_name):
    return table_scans[table_scans['TABLE_NAME'] == table_name]

def get_table_names(table_scans):
    return table_scans['TABLE_NAME'].unique()

def extract_probe_side_joins(joins, table_name):
    return joins[joins['PROBE_TABLE'] == table_name]


def default_benchmark_config():    
    if BENCHMARK == "TPCH":
        config = {
            'lineitem': [['l_shipdate', 92 * SCALE_FACTOR]],
            'orders': [['o_orderdate', 23 * SCALE_FACTOR]]
        }
    elif BENCHMARK == "TPCDS":
        config = dict()
    else:        
        raise Exception("unknown benchmark, please provide a default config")
    return config

def get_correlations():
    if BENCHMARK == "TPCH":
        correlations = {
            'lineitem': {
                'l_shipdate': ['l_receiptdate', 'l_commitdate'],
                'l_receiptdate': ['l_shipdate', 'l_commitdate'],
            }
        }
    elif BENCHMARK == "TPCDS":
        correlations = dict()
    else:
        raise Exception("unknown benchmark, please provide correlation information")
        
    return correlations


def format_table_clustering(clustering_config):
    # input format: List of [ [(column, split)+ ], sorting_column, runtime ]
    # output format: List of [ (column, split)+ ] - sorting column integrated if necessary
    
    assert len(clustering_config) == 3, "config should have exactly three entries: clustering columns, sort column, runtime"
    clustering_columns = clustering_config[0]
    assert len(clustering_columns) <= 3, "atm the model is at most 3-dimensional"
    #print(f"clustering columns are {clustering_columns}")
    last_clustering_column = clustering_columns[-1]
    last_clustering_column_name = last_clustering_column[0]
    #print(f"last column is {last_clustering_column_name}")
    sorting_column = clustering_config[1]
    #print(f"sort column is {sorting_column}")
    
    result = clustering_columns
    if last_clustering_column_name != sorting_column:
        result = clustering_columns + [(sorting_column, 1)]
        
    #print(f"in: {clustering_config}")
    #print(f"out: {result}")
    
    return result

def get_config_name(clustering_config):
    # Input: config-dict
    
    # List of lists. Each secondary list contains clustering information for a table
    table_configs = [clustering_config[table] for table in clustering_config]
    config_entries = [[f"{config_entry[0]}-{config_entry[1]}" for config_entry in config] for config in table_configs]
    table_entries = ["_".join(config) for config in config_entries]
    return "_".join(table_entries)


def create_benchmark_configs():
    
    start_time = datetime.now()
    clusterings = {"default" : default_benchmark_config()}
    query_frequencies = get_query_frequencies()
    
    distinct_values = get_distinct_values_count()
    joins = load_join_statistics()    
    sorted_columns_during_creation = get_sorted_columns_during_creation()
    correlations = get_correlations()
    table_names = get_table_names(scans)
    for table_name in table_names:
        start_time_table = datetime.now()
        single_table_scans = extract_single_table(scans, table_name)
        probe_side_joins = joins#extract_probe_side_joins(joins, table_name)
        table_size = table_sizes[table_name]
        if table_size <= 3 * CHUNK_SIZE:
            print(f"Not computing clustering for {table_name}, as it has only {table_size} rows")
            continue

        model = PartitionerModel(query_frequencies, table_name, single_table_scans, table_size, distinct_values[table_name], CHUNK_SIZE, correlations.get(table_name, {}), probe_side_joins, sorted_columns_during_creation)
        table_clusterings = model.suggest_clustering(3)
        for table_clustering in table_clusterings:
            config = default_benchmark_config()
            config[table_name] = format_table_clustering(table_clustering)
            config_name = get_config_name(config)
            clusterings[config_name] = config
        end_time_table = datetime.now()
        print(f"Done computing clustering for {table_name} ({end_time_table - start_time_table})")

            
    end_time = datetime.now()
    print(f"Computed all clusterings in {end_time - start_time}")
    
    return clusterings

create_benchmark_configs()

# TODO:
#  joins costs are multiplied with 0
#  still, the model suggests some join columns - why? are they useful for pruning?

In [34]:
class SingleTableMdcModel(AbstractModel):
    
    def __init__(self, max_dimensions, query_frequencies, table_name, table_scans, table_sizes, distinct_values, target_chunksize, correlations, joins, sorted_columns_during_creation):
        super().__init__(query_frequencies, table_name, table_scans, correlations)
        self.max_dimensions = max_dimensions
        self.table_sizes = table_sizes       
        self.table_size = table_sizes[table_name]
        self.distinct_values = distinct_values
        self.target_chunksize = target_chunksize
        self.joins = joins
        self.sorted_columns_during_creation = sorted_columns_during_creation
        
        self.join_column_names = self.extract_join_columns()
        self.scan_column_names = self.extract_scan_columns()
        
    def is_join_column(self, column_name):
        return column_name in self.join_column_names
    
    def is_scan_column(self, column_name):
        return column_name in self.scan_column_names
    
    def suggest_clustering(self, first_k=1):
        interesting_columns = self.extract_interesting_columns()

        print(interesting_columns)
        clustering_columns = itertools.combinations_with_replacement(interesting_columns, self.max_dimensions)
        clustering_columns = [self.uniquify(clustering) for clustering in clustering_columns]
        sort_columns = interesting_columns        
        clusterings_with_runtimes = reduce(lambda x,y: x+y,[self.estimate_total_runtimes(clustering_cols, sort_columns) for clustering_cols in clustering_columns])
        clusterings_with_runtimes.sort(key=lambda x: x[2], reverse=False)
        
        return clusterings_with_runtimes[0:first_k]
    
    
    def estimate_distinct_values_per_chunk(self, column, clustering_columns, sorting_column, dimension_cardinalities):
        raise NotImplementedError("Each model should provide this function")
        
    def estimate_distinct_values_per_chunk_at_statistics_time(self, column, table):        
        if column in self.sorted_columns_during_creation.get(table, {}):
            # Column was globally sorted
            average_count_per_distinct_value = self.table_sizes[table] / self.distinct_values[table][column]
            return math.ceil(self.target_chunksize / average_count_per_distinct_value)
        else:
            # Column was not globally sorted
            total_distinct_values = self.distinct_values[table][column]
            return min(total_distinct_values, self.target_chunksize)        
    
    def estimate_table_scan_runtime(self, clustering_columns, sorting_column, split_factors):
        def compute_unprunable_parts(row, split_factors):
            def clustering_columns_correlated_to(column):
                return [clustering_column for clustering_column in clustering_columns if column in self.correlations.get(clustering_column, {})]
            
            def correlates_to_clustering_column(column):
                return len(clustering_columns_correlated_to(column)) > 0

            column_name = row['COLUMN_NAME']

            if not row['useful_for_pruning']:
                selectivity = 1
            elif column_name in clustering_columns:
                scan_selectivity = row['selectivity']
                split_factor = split_factors[clustering_columns.index(column_name)]
                selectivity =  self.round_up_to_next_multiple(scan_selectivity, 1 / split_factor)
            elif correlates_to_clustering_column(column_name):
                scan_selectivity = row['selectivity']
                correlated_clustering_columns = clustering_columns_correlated_to(column_name)
                
                # ToDo this is hacky, but for now assume there is just one correlated column
                assert len(correlated_clustering_columns) == 1, f"expected just 1 correlated clustering column, but got {len(correlated_clustering_columns)}"
                
                split_factor = split_factors[clustering_columns.index(correlated_clustering_columns[0])]
                selectivity = min(1, 1.2 * self.round_up_to_next_multiple(scan_selectivity, 1 / split_factor))
            else:
                selectivity = 1
            
            return selectivity
        
        def compute_tablescan_runtime(row, sorting_column):
            assert row['estimated_input_rows'] > 1, row
            assert row['runtime_per_input_row'] > 0, row
            assert row['runtime_per_output_row'] > 0, row
            input_row_count = row['estimated_input_rows']
            
            if row['COLUMN_NAME'] == sorting_column and row['benefits_from_sorting']:
                # TODO is this the best way to simulate sorted access?
                input_row_count = np.log2(input_row_count)

            runtime = input_row_count * row['runtime_per_input_row'] + row['OUTPUT_ROWS'] * row['runtime_per_output_row']
            return runtime * self.query_frequency(row['QUERY_HASH'])
        
        runtime = 0
        scans_per_query = self.table_scans.sort_values(['INPUT_ROWS'], ascending=False).groupby(['QUERY_HASH', 'GET_TABLE_HASH'])
        for _, scans in scans_per_query:
            number_of_scans = len(scans)
            assert number_of_scans > 0 and number_of_scans < 25, f"weird scan length: {number_of_scans}\nScans:\n{scans}"
            # TODO: kinda unrealistic assumption: everything not in the table scan result can be pruned
                          
            unprunable_parts = scans.apply(compute_unprunable_parts, axis=1, args=(split_factors,))            
            unprunable_part = unprunable_parts.product()
            assert unprunable_part > 0, "no unprunable part"
            
            estimated_pruned_table_size = self.round_up_to_next_multiple(unprunable_part * self.table_size, CHUNK_SIZE)
            
            runtimes = pd.DataFrame()
            runtimes['QUERY_HASH'] = scans['QUERY_HASH']
            runtimes['runtime_per_input_row'] = scans['time_per_input_row']
            runtimes['runtime_per_output_row'] = scans['time_per_output_row']
            runtimes['COLUMN_NAME'] = scans['COLUMN_NAME']
            runtimes['benefits_from_sorting'] = scans['benefits_from_sorting']
            # the pruned table inputs should be reflected in 'estimated_input_rows'
            runtimes['estimated_input_rows'] = scans['INPUT_ROWS']
            runtimes['OUTPUT_ROWS'] = scans['OUTPUT_ROWS']

            runtimes.iloc[0, runtimes.columns.get_loc('estimated_input_rows')] = estimated_pruned_table_size                                    
            assert runtimes['estimated_input_rows'].iloc[0] == estimated_pruned_table_size, f"value is {runtimes.iloc[0]['estimated_input_rows']}, but should be {estimated_pruned_table_size}"
            # TODO modify input sizes of subsequent scans
            
            scan_runtimes = runtimes.apply(compute_tablescan_runtime, axis=1, args=(sorting_column,))
            runtime += scan_runtimes.sum()
        return runtime

    def estimate_chunk_count(self, scans, clustering_columns, dimension_cardinalities):
        raise NotImplementedError("Subclass responsibility")
    
    def get_chunk_count_factor(self, row, side, clustering_columns, dimension_cardinalities):
        query_hash = row['QUERY_HASH']
        if side == "PROBE":
            input_rows = row['PROBE_TABLE_ROW_COUNT']
            assert row['PROBE_TABLE'] == self.table_name, "Call this function only for the own table"
        elif side == "BUILD":
            input_rows = row['BUILD_TABLE_ROW_COUNT']
            assert row['BUILD_TABLE'] == self.table_name, "Call this function only for the own table"
        else:
            raise ValueError("side must be PROBE or BUILD")

        expected_chunk_count = self.estimate_chunk_count(query_hash, input_rows, clustering_columns, dimension_cardinalities)
        min_chunk_count = math.ceil(input_rows / self.target_chunksize)
        max_chunk_count = math.ceil(self.table_size / self.target_chunksize)

        CHUNK_COUNT_SPEEDUP_LOW = 3
        CHUNK_COUNT_SPEEDUP_HIGH = 1

        current_speedup = self.interpolate(CHUNK_COUNT_SPEEDUP_LOW, CHUNK_COUNT_SPEEDUP_HIGH, (expected_chunk_count - min_chunk_count) / max_chunk_count)
        #print(f"current speedup: {current_speedup}")

        old_clustering_columns = self.sorted_columns_during_creation[self.table_name]
        old_dimension_cardinalities = [self.statistic_time_dimension_cardinalities()] * len(old_clustering_columns)
        old_expected_chunk_count = self.estimate_chunk_count(query_hash, input_rows, old_clustering_columns, old_dimension_cardinalities)
        old_speedup = self.interpolate(CHUNK_COUNT_SPEEDUP_LOW, CHUNK_COUNT_SPEEDUP_HIGH, (old_expected_chunk_count - min_chunk_count) / max_chunk_count)
        #print(f"old speedup: {old_speedup}")

        return 1 * old_speedup / current_speedup
    
    def estimate_join_runtime(self, clustering_columns, sorting_column, dimension_cardinalities):
        def compute_join_runtime(row, sorting_column):
            if "JoinHash" in row['DESCRIPTION']:
                probe_column = row['PROBE_COLUMN']
                if row['PROBE_TABLE'] == self.table_name:
                    probe_column_was_sorted = row['PROBE_SORTED'] and probe_column in self.sorted_columns_during_creation.get(self.table_name, {})
                    probe_column_is_sorted = row['PROBE_SORTED'] and probe_column == sorting_column
                    materialize_probe_factor = self.get_chunk_count_factor(row, "PROBE", clustering_columns, dimension_cardinalities)
                else:
                    probe_column_was_sorted = row['PROBE_SORTED'] and probe_column in self.sorted_columns_during_creation.get(row['PROBE_TABLE'], {})
                    probe_column_is_sorted = probe_column_was_sorted
                    materialize_probe_factor = 1
                    
                build_column = row['BUILD_COLUMN']
                if row['BUILD_TABLE'] == self.table_name:
                    build_column_was_sorted = row['BUILD_SORTED'] and build_column in self.sorted_columns_during_creation.get(self.table_name, {})
                    build_column_is_sorted = row['BUILD_SORTED'] and build_column == sorting_column
                    materialize_build_factor = self.get_chunk_count_factor(row, "BUILD", clustering_columns, dimension_cardinalities)
                else:
                    build_column_was_sorted = row['BUILD_SORTED'] and build_column in self.sorted_columns_during_creation.get(row['BUILD_TABLE'], {})
                    build_column_is_sorted = build_column_was_sorted
                    materialize_build_factor = 1

                time_materialize_probe = row['PROBE_SIDE_MATERIALIZING_NS']
                time_materialize_build = row['BUILD_SIDE_MATERIALIZING_NS']
                time_materialize = time_materialize_build + time_materialize_probe

                probe_weight = 2
                build_weight = 2
                if probe_column_was_sorted:
                    probe_weight = 1
                if build_column_was_sorted:
                    build_weight = 1

                probe_table_size = row['PROBE_TABLE_ROW_COUNT']
                build_table_size = row['BUILD_TABLE_ROW_COUNT']
                total_table_size = probe_weight * probe_table_size + build_weight * build_table_size

                time_materialize_probe = time_materialize * (probe_weight * probe_table_size / total_table_size)
                time_materialize_build = time_materialize - time_materialize_probe
                
                def get_materialize_factor(column, was_globally_sorted, is_sorted, expected_distinct_value_count):
                    # Assumption: "was_sorted" implies global sortedness, i.e., both clustering and chunkwise sorting
                    # This is true when the clustering produced by the table generator is used by the plan cache exporter
                    # If the data has been re-clustered before the plan cache exporter runs, there has to be some system inside Hyrise which tracks the current clustering config
                    
                    # Sortedness seems to yield a speed up of approx. 1.6, regardless of the number of distinct values
                    SORT_SPEEDUP = 1.6
                    sortedness_factor = 1                    
                    was_sorted = was_globally_sorted
                    if was_sorted:
                        sortedness_factor *= SORT_SPEEDUP
                    if is_sorted:
                        sortedness_factor /= SORT_SPEEDUP
                    

                    # The influence of clustering depends on the number of distinct values
                    CLUSTERING_SPEEDUP_LOW = 1.84
                    CLUSTERING_SPEEDUP_HIGH = 1
                    clustering_factor = 1
                    statistics_time_distinct_value_count = self.estimate_distinct_values_per_chunk_at_statistics_time(column, self.table_name);
                    clustering_factor *= self.interpolate(CLUSTERING_SPEEDUP_LOW, CLUSTERING_SPEEDUP_HIGH, statistics_time_distinct_value_count / self.target_chunksize)
                    clustering_factor /= self.interpolate(CLUSTERING_SPEEDUP_LOW, CLUSTERING_SPEEDUP_HIGH, expected_distinct_value_count / self.target_chunksize)
                        
                    return sortedness_factor * clustering_factor                
                
                if row['PROBE_TABLE'] == self.table_name and row['PROBE_SORTED']:
                    expected_distinct_values_probe = self.estimate_distinct_values_per_chunk(probe_column, clustering_columns, sorting_column, dimension_cardinalities)
                    materialize_probe_factor *= get_materialize_factor(probe_column, probe_column_was_sorted, probe_column_is_sorted, expected_distinct_values_probe)
                    
                if row['BUILD_TABLE'] == self.table_name and row['BUILD_SORTED']:
                    expected_distinct_values_build = self.estimate_distinct_values_per_chunk(build_column, clustering_columns, sorting_column, dimension_cardinalities)
                    materialize_build_factor *= get_materialize_factor(build_column, build_column_was_sorted, build_column_is_sorted, expected_distinct_values_build)                
                
                time_materialize = time_materialize_probe * materialize_probe_factor + time_materialize_build *  materialize_build_factor


                # unchanged
                time_cluster = row['CLUSTERING_NS']
                
                # unchanged
                time_build = row['BUILDING_NS']
                
                            
                time_probe = row['PROBING_NS']
                probe_factor = 1
                #if probe_column_is_sorted and probe_column_is_clustered:
                #    if not probe_column_was_sorted:
                #        probe_factor = 0.7
                #    else:
                #        probe_factor = 1
                #elif probe_column_is_sorted or probe_column_is_clustered:
                #    if not probe_column_was_sorted:
                #        probe_factor = 0.9
                #    else:
                #        probe_factor = 1.1
                #elif probe_column_was_sorted:
                #    # probe column is now neither sorted nor clustered
                #    probe_factor = 1.4
                
                time_probe *= probe_factor                
                
                # unchanged
                time_write_output = row['OUTPUT_WRITING_NS']
                
                
                
                # TODO: how to deal with the difference between RUNTIME_NS and sum(stage_runtimes)?
                runtime = time_materialize + time_cluster + time_build + time_probe + time_write_output
            else:
                runtime = row['RUNTIME_NS']
                
            return runtime * self.query_frequency(row['QUERY_HASH'])
        
        join_runtimes = self.joins.apply(compute_join_runtime, axis=1, args=(sorting_column,))
        return join_runtimes.sum()
                
    def estimate_total_runtimes(self, clustering_columns, sorting_columns):
        #print(f"testing clustering {clustering_columns} with sorting columns {sorting_columns}")
        dimension_cardinalities = self.get_dimension_cardinalities(clustering_columns)
        total_runtimes = {sorting_column: 0 for sorting_column in sorting_columns}
        clusterings = []
        for sorting_column in sorting_columns:
            runtime = self.estimate_total_runtime(clustering_columns, sorting_column, dimension_cardinalities)
            clusterings.append([list(zip(clustering_columns, dimension_cardinalities)), sorting_column, runtime])
        
        #clusterings = [[list(zip(clustering_columns, dimension_cardinalities)), sorting_column, np.int64(total_runtimes[sorting_column])] for sorting_column in sorting_columns]
        return clusterings
    
    def estimate_total_runtime(self, clustering_columns, sorting_column, dimension_cardinalities):
        runtime = 0
        runtime += self.estimate_table_scan_runtime(clustering_columns, sorting_column, dimension_cardinalities)
        runtime += self.estimate_join_runtime(clustering_columns, sorting_column, dimension_cardinalities)
        
        return runtime
    
    def get_dimension_cardinalities(self, clustering_columns):
        raise NotImplementedError("Subclasses must override this function")
        
    def statistic_time_dimension_cardinalities(self):
        raise NotImplementedError("Subclasses must override this function")
        
    def interpolate(self, low, high, percentage):
        assert percentage >= 0 and percentage <= 1, f"percentage must between 0 and 1, but is {percentage}"
        return (1 - percentage) * low + (percentage * high)

In [35]:
class DisjointClustersModel(SingleTableMdcModel):
    
    def __init__(self, max_dimensions, query_frequencies, table_name, table_scans, table_sizes, distinct_values, target_chunksize, correlations, joins, sorted_columns_during_creation):
        super().__init__(max_dimensions, query_frequencies, table_name, table_scans, table_sizes, distinct_values, target_chunksize, correlations, joins, sorted_columns_during_creation)
        
    # This function should only be called for the own table, not for others
    def estimate_distinct_values_per_chunk(self, column, clustering_columns, chunk_sorting_column, dimension_cardinalities):
        total_distinct_values = self.distinct_values[self.table_name][column]
        
        if column in clustering_columns:
            index = clustering_columns.index(column)
            clusters_for_column = dimension_cardinalities[index]
            return min(self.target_chunksize, total_distinct_values / clusters_for_column)
        else:
            # TODO cluster wise sorting
            return min(self.target_chunksize, total_distinct_values)
        
    # This function should only be called for the own table, not for others
    def estimate_chunk_count(self, query_hash, join_input_rows, clustering_columns, dimension_cardinalities):        
        # TODO include or exclude scans that do not benefit from pruning? Included for now        
        table_scans = self.table_scans[self.table_scans['QUERY_HASH'] == query_hash]        
        table_scans = table_scans[table_scans['useful_for_pruning']]
        
        if len(table_scans) > 0:
            #print(f"For query hash {query_hash}, there are {len(table_scans)} scans on {self.table_name}")
            def get_denseness_factor(row):
                column = row['COLUMN_NAME']
                if column in clustering_columns:
                    # TODO more precise estimate
                    denseness_factor = 1
                else:
                    denseness_factor = row['selectivity']

                return denseness_factor

            denseness_factors = table_scans.apply(get_denseness_factor, axis=1)
            denseness_factor = denseness_factors.product()
            #print(f"denseness factor is {denseness_factor}")
        else:
            denseness_factor = 1        
        
        chunk_count = math.ceil(join_input_rows / (self.target_chunksize * denseness_factor))
        max_chunks = math.ceil(self.table_size / self.target_chunksize)
        assert chunk_count <= max_chunks, f"estimated {chunk_count} chunks, but {self.table_name} got only {max_chunks}\nDenseness: {denseness_factor}"
        
        return chunk_count
    
    def statistic_time_dimension_cardinalities(self):
        return math.ceil(self.table_size / self.target_chunksize)
            
    def get_dimension_cardinalities(self, clustering_columns):
        # ToDo what if we aim at less than number of chunks clusters, i.e. multiple chunks per cluster?
        target_cluster_count = int(1.1 * self.table_size / self.target_chunksize)
        # idea: fixed size for join columns, variable amount for scan columns
        
        join_columns = list(filter(lambda x: self.is_join_column(x), clustering_columns))
        scan_columns = list(filter(lambda x: self.is_scan_column(x), clustering_columns))
        intersecting_columns = set(join_columns).intersection(set(scan_columns))
        assert len(intersecting_columns) == 0, f"The following columns are used as both join and scan column: {intersecting_columns}"
        
        if len(scan_columns) == 0:
            CLUSTERS_PER_JOIN_COLUMN = math.ceil(math.pow(target_cluster_count, 1/len(join_columns)))
        else: 
            CLUSTERS_PER_JOIN_COLUMN = 3;
        # Assumption: uniform distribution (in the sense that every cluster actually exists)
        num_join_clusters = math.pow(CLUSTERS_PER_JOIN_COLUMN, len(join_columns))
        assert num_join_clusters <= 1.5 * target_cluster_count, f"Would get {num_join_clusters} clusters for join columns, but aimed at at most {target_cluster_count} clusters"
    
        # only applies to scan columns
        desired_scan_clusters_count = math.ceil(target_cluster_count / num_join_clusters)
        individual_distinct_values = [self.distinct_values[self.table_name][column] for column in scan_columns]
        log_distinct_values = [math.ceil(0.5+np.log2(x)) for x in individual_distinct_values]
        log_distinct_values_product = reduce(operator.mul, log_distinct_values, 1)
        assert log_distinct_values_product > 0, "cannot have a distinct value count of 0"

        global_modification_factor = desired_scan_clusters_count / log_distinct_values_product
        num_scan_dimensions = len(scan_columns)
        individual_modification_factor = np.power(global_modification_factor, 1.0 / max(1, num_scan_dimensions))
        
        join_column_cluster_counts = [CLUSTERS_PER_JOIN_COLUMN] * len(join_columns)
        scan_column_cluster_counts = [math.ceil(x * individual_modification_factor) for x in log_distinct_values]
        
        
        # Merge join and scan columns
        join_index = 0
        scan_index = 0
        cluster_counts = []
        for clustering_column in clustering_columns:
            if clustering_column in join_columns:
                cluster_counts.append(join_column_cluster_counts[join_index])
                join_index += 1
            elif clustering_column in scan_columns:
                cluster_counts.append(scan_column_cluster_counts[scan_index])
                scan_index += 1
        assert join_index == len(join_columns), f"Processed the wrong number of join columns: {join_index} instead of {len(join_column_cluster_counts)}"
        assert scan_index == len(scan_columns), f"Processed the wrong number of scan columns: {scan_index} instead of {len(scan_column_cluster_counts)}"
        assert len(cluster_counts) == len(clustering_columns), f"Expected {len(clustering_columns)} cluster counts, but got {len(cluster_counts)}"
        
        # testing
        actual_cluster_count = reduce(operator.mul, cluster_counts, 1)
        assert actual_cluster_count > 0, "there was a split up factor of 0"
        assert actual_cluster_count <= 1.5 * target_cluster_count, f"Wanted at most {target_cluster_count} clusters, but got {actual_cluster_count}\nConfig: {clustering_columns}\nCluster sizes: {cluster_counts}"
        estimated_chunksize = self.table_size / actual_cluster_count
        assert estimated_chunksize <= self.target_chunksize, "chunks should be smaller, not larger than target_chunksize"
        allowed_percentage = 0.55
        if estimated_chunksize < allowed_percentage * self.target_chunksize:
            print(f"Warning: chunks should not be too much smaller than target_chunksize: {estimated_chunksize} < {allowed_percentage} * {self.target_chunksize}")
        #assert estimated_chunksize >= allowed_percentage * self.target_chunksize, f"chunks should not be too much smaller than target_chunksize: {estimated_chunksize} < {allowed_percentage} * {self.target_chunksize}"
        
        return cluster_counts

In [37]:
assert_correct_statistics_loaded()

def extract_single_table(table_scans, table_name):
    return table_scans[table_scans['TABLE_NAME'] == table_name]

def get_table_names(table_scans):
    return table_scans['TABLE_NAME'].unique()


def default_benchmark_config():    
    if BENCHMARK == "TPCH":
        config = {
            'lineitem': [['l_shipdate', 2]],
            'orders': [['o_orderdate', 2]]
        }
    elif BENCHMARK == "TPCDS":
        config = dict()
    else:        
        raise Exception("unknown benchmark, please provide a default config")
    return config

def get_correlations():
    if BENCHMARK == "TPCH":
        correlations = {
            'lineitem': {
                'l_shipdate': ['l_receiptdate', 'l_commitdate'],
                'l_receiptdate': ['l_shipdate', 'l_commitdate'],
            }
        }
    elif BENCHMARK == "TPCDS":
        correlations = dict()
    else:
        raise Exception("unknown benchmark, please provide correlation information")
        
    return correlations


def format_table_clustering(clustering_config):
    # input format: List of [ [(column, split)+ ], sorting_column, runtime ]
    # output format: List of [ (column, split)+ ] - sorting column integrated if necessary
    
    assert len(clustering_config) == 3, "config should have exactly three entries: clustering columns, sort column, runtime"
    clustering_columns = clustering_config[0]
    assert len(clustering_columns) <= 3, "atm the model is at most 3-dimensional"
    #print(f"clustering columns are {clustering_columns}")
    last_clustering_column = clustering_columns[-1]
    last_clustering_column_name = last_clustering_column[0]
    #print(f"last column is {last_clustering_column_name}")
    sorting_column = clustering_config[1]
    #print(f"sort column is {sorting_column}")
    
    result = clustering_columns
    if last_clustering_column_name != sorting_column:
        result = clustering_columns + [(sorting_column, 1)]
        
    #print(f"in: {clustering_config}")
    #print(f"out: {result}")
    
    return result

def get_config_name(clustering_config):
    # Input: config-dict
    
    # List of lists. Each secondary list contains clustering information for a table
    table_configs = [clustering_config[table] for table in clustering_config]
    config_entries = [[f"{config_entry[0]}-{config_entry[1]}" for config_entry in config] for config in table_configs]
    table_entries = ["_".join(config) for config in config_entries]
    return "_".join(table_entries)


def create_model(table_name, max_dimensions=2):    
    query_frequencies = get_query_frequencies()
    distinct_values = get_distinct_values_count()
    joins = load_join_statistics()    
    sorted_columns_during_creation = get_sorted_columns_during_creation()
    correlations = get_correlations()
    table_names = get_table_names(scans)
    start_time_table = datetime.now()
    single_table_scans = extract_single_table(scans, table_name)

    model = DisjointClustersModel(max_dimensions, query_frequencies, table_name, single_table_scans, table_sizes, distinct_values, CHUNK_SIZE, correlations.get(table_name, {}), joins, sorted_columns_during_creation)
    return model

model = create_model("lineitem", 2)

In [40]:

#model.suggest_clustering(20)
#model.estimate_distinct_values_per_chunk_at_statistics_time("l_orderkey", "lineitem")
#print(model.estimate_distinct_values_per_chunk("l_orderkey", ["l_orderkey", "l_partkey"], "l_orderkey", [20,5]))

#model.estimate_chunk_count("9762c3a887e47469", 77313, ["l_orderkey"], [92])
#row = model.joins.iloc[140]
#model.get_chunk_count_factor(row, "BUILD", ["l_orderkey", "l_shipdate"], [5, 20])


time0 = model.estimate_total_runtime(["l_orderkey"], "l_orderkey", [100])
time1 = model.estimate_total_runtime(["l_shipdate", "l_orderkey"], "l_orderkey", [20, 5])
print(f"nosort: {time0}")
print(f"best: {time1}")

nosort: 26277130840.6223
best: 26780150765.066814


In [41]:
start_time = datetime.now()
clusterings = model.suggest_clustering(20)
end_time = datetime.now()
print(f"Execution time: {end_time - start_time}")
clusterings

['l_receiptdate', 'l_shipdate', 'l_quantity', 'l_discount', 'l_partkey', 'l_suppkey', 'l_orderkey']
Execution time: 0:06:08.220281


[[[('l_partkey', 100)], 'l_orderkey', 26274868604.625267],
 [[('l_orderkey', 100)], 'l_orderkey', 26277130840.6223],
 [[('l_orderkey', 100)], 'l_partkey', 26366046972.651962],
 [[('l_partkey', 10), ('l_suppkey', 10)], 'l_orderkey', 26450487210.97882],
 [[('l_partkey', 10), ('l_orderkey', 10)], 'l_orderkey', 26454139615.61673],
 [[('l_orderkey', 100)], 'l_shipdate', 26623966875.35761],
 [[('l_shipdate', 18), ('l_discount', 6)], 'l_orderkey', 26755818292.507927],
 [[('l_shipdate', 100)], 'l_orderkey', 26759861849.49215],
 [[('l_shipdate', 34), ('l_suppkey', 3)], 'l_orderkey', 26769456824.93312],
 [[('l_shipdate', 34), ('l_partkey', 3)], 'l_orderkey', 26772207330.6715],
 [[('l_shipdate', 34), ('l_orderkey', 3)], 'l_orderkey', 26772207330.6715],
 [[('l_shipdate', 14), ('l_quantity', 8)], 'l_orderkey', 26773777261.658085],
 [[('l_receiptdate', 10), ('l_shipdate', 10)],
  'l_orderkey',
  26786686919.74545],
 [[('l_receiptdate', 18), ('l_discount', 6)],
  'l_orderkey',
  26795330857.347076],


In [ ]:
assert_correct_statistics_loaded()

def extract_single_table(table_scans, table_name):
    return table_scans[table_scans['TABLE_NAME'] == table_name]

def get_table_names(table_scans):
    return table_scans['TABLE_NAME'].unique()


def default_benchmark_config():    
    if BENCHMARK == "TPCH":
        config = {
            'lineitem': [['l_shipdate', 2]],
            'orders': [['o_orderdate', 2]]
        }
    elif BENCHMARK == "TPCDS":
        config = dict()
    else:        
        raise Exception("unknown benchmark, please provide a default config")
    return config

def get_correlations():
    if BENCHMARK == "TPCH":
        correlations = {
            'lineitem': {
                'l_shipdate': ['l_receiptdate', 'l_commitdate'],
                'l_receiptdate': ['l_shipdate', 'l_commitdate'],
            }
        }
    elif BENCHMARK == "TPCDS":
        correlations = dict()
    else:
        raise Exception("unknown benchmark, please provide correlation information")
        
    return correlations


def format_table_clustering(clustering_config):
    # input format: List of [ [(column, split)+ ], sorting_column, runtime ]
    # output format: List of [ (column, split)+ ] - sorting column integrated if necessary
    
    assert len(clustering_config) == 3, "config should have exactly three entries: clustering columns, sort column, runtime"
    clustering_columns = clustering_config[0]
    assert len(clustering_columns) <= 3, "atm the model is at most 3-dimensional"
    #print(f"clustering columns are {clustering_columns}")
    last_clustering_column = clustering_columns[-1]
    last_clustering_column_name = last_clustering_column[0]
    #print(f"last column is {last_clustering_column_name}")
    sorting_column = clustering_config[1]
    #print(f"sort column is {sorting_column}")
    
    result = clustering_columns
    if last_clustering_column_name != sorting_column:
        result = clustering_columns + [(sorting_column, 1)]
        
    #print(f"in: {clustering_config}")
    #print(f"out: {result}")
    
    return result

def get_config_name(clustering_config):
    # Input: config-dict
    
    # List of lists. Each secondary list contains clustering information for a table
    table_configs = [clustering_config[table] for table in clustering_config]
    config_entries = [[f"{config_entry[0]}-{config_entry[1]}" for config_entry in config] for config in table_configs]
    table_entries = ["_".join(config) for config in config_entries]
    return "_".join(table_entries)


def create_benchmark_configs():    
    start_time = datetime.now()
    clusterings = {"default" : default_benchmark_config()}
    query_frequencies = get_query_frequencies()
    
    distinct_values = get_distinct_values_count()
    joins = load_join_statistics()    
    sorted_columns_during_creation = get_sorted_columns_during_creation()
    correlations = get_correlations()
    table_names = get_table_names(scans)
    for table_name in table_names:
        start_time_table = datetime.now()
        single_table_scans = extract_single_table(scans, table_name)
        table_size = table_sizes[table_name]
        if table_size <= 3 * CHUNK_SIZE:
            print(f"Not computing clustering for {table_name}, as it has only {table_size} rows")
            continue

        max_dimensions = 2
        model = DisjointClustersModel(max_dimensions, query_frequencies, table_name, single_table_scans, table_sizes, distinct_values, CHUNK_SIZE, correlations.get(table_name, {}), joins, sorted_columns_during_creation)
        table_clusterings = model.suggest_clustering(3)
        for table_clustering in table_clusterings:
            config = default_benchmark_config()
            config[table_name] = format_table_clustering(table_clustering)
            config_name = get_config_name(config)
            clusterings[config_name] = config
        end_time_table = datetime.now()
        print(f"Done computing clustering for {table_name} ({end_time_table - start_time_table})")
    
    end_time = datetime.now()
    print(f"Computed all clusterings in {end_time - start_time}")
    
    return clusterings

create_benchmark_configs()

# TODO:
#  joins costs are multiplied with 0
#  still, the model suggests some join columns - why? are they useful for pruning?

Outdated code fragments (older model versions) are kept below.

In [ ]:
assert False, "This assertion failure only serves to stop execution here when clicking Cells->Run all. You can safely ignore it."

In [ ]:
class SimpleModel(AbstractModel):
    
    def __init__(self, table_scans, correlations = {}):
        super().__init__(table_scans, correlations)        
    
    def suggest_clustering(self, first_k=1):
        interesting_columns = self.extract_interesting_columns()

        pairs = itertools.product(interesting_columns, interesting_columns)                
        total_runtimes = [self.estimate_total_runtime(self.table_scans, clustering_columns) for clustering_columns in pairs]
        total_runtimes.sort(key=lambda x: x[1], reverse=False)
        
        return total_runtimes[0:first_k]
        
    
    def estimate_total_runtime(self, single_table, clustering_columns):
        total_runtime = 0
        
        pruning_col = clustering_columns[0]
        sorted_col = clustering_columns[1]
        def compute_runtime(row):
            col_name = row['COLUMN_NAME']
            if pruning_col == sorted_col:
                if col_name == pruning_col:
                    return row['optimal_log_runtime']
                else:
                    if col_name in self.correlations.get(pruning_col, []):
                        # correlated to pruning column -> a lot of pruning, no sortedness
                        # TODO: better measure correlation
                        return 1.2 * row['optimal_runtime']
                    else:
                        return row['RUNTIME_NS']

            else:
                if col_name == pruning_col:
                    return row['optimal_runtime']
                elif col_name == sorted_col:
                    # TODO: should this be affected by correlation?
                    # we will get less chunks, so a linear scan should be close to optimal_runtime,
                    # but log time should beat it anyway
                    return row['log_runtime']
                else:
                    if col_name in self.correlations.get(pruning_col, []):
                        # correlated to pruning column -> a lot of pruning, no sortedness
                        # TODO: better measure correlation
                        return 1.2 * row['optimal_runtime']
                    else:
                        return row['RUNTIME_NS']
                    
        effective_runtime = single_table.apply(compute_runtime, axis=1)
        return [clustering_columns, effective_runtime.sum()]

In [ ]:
# Store additional statistics
# TODO keep?

assert_correct_statistics_loaded()

def round_up_to_chunksize(row):
    if row['OUTPUT_ROWS'] % CHUNK_SIZE == 0:
        return row['OUTPUT_ROWS']
    else:
        return row['OUTPUT_ROWS'] + (CHUNK_SIZE - (row['OUTPUT_ROWS'] % CHUNK_SIZE))

scans['pruned_minimum_input_rows'] = scans.apply(round_up_to_chunksize, axis=1)

scans['selectivity'] = scans['OUTPUT_ROWS'] / scans['INPUT_ROWS']
scans['actual_selectivity'] = scans['SINGLE_OUTPUT_ROWS'] / scans['SINGLE_INPUT_ROWS']

scans['time_per_ir'] = scans['RUNTIME_NS'] / scans['INPUT_ROWS']
scans['time_per_or'] = scans['RUNTIME_NS'] / scans['OUTPUT_ROWS']

# optimal runtime assuming perfect pruning, but not sortedness
scans['optimal_runtime'] = scans['time_per_ir'] * scans['pruned_minimum_input_rows']
scans['runtime_gain'] = scans['RUNTIME_NS'] - scans['optimal_runtime']


# log runtime for sorted columns
scans['log_runtime'] = np.log2(scans['RUNTIME_NS'])
scans['optimal_log_runtime'] = np.log2(1+scans['optimal_runtime'])
scans

In [ ]:
GAIN_COLUMN = 'runtime_gain'

scans_groupby_columnname = scans.groupby(['TABLE_NAME', 'COLUMN_NAME'])
sum_of_gains = pd.DataFrame(scans_groupby_columnname[GAIN_COLUMN].sum())
sum_of_gains.sort_values(by=['TABLE_NAME', GAIN_COLUMN], ascending=[True, False])

In [ ]:
assert_correct_statistics_loaded()

if BENCHMARK == "TPCH":
    TABLE = "lineitem"
else:    
    TABLE = "customer_demographics"

import itertools

def extract_single_table(table_name):
    return scans[scans['TABLE_NAME'] == table_name]

def extract_interesting_columns(df):
    return list(df['COLUMN_NAME'].unique())


correlations = {
    'l_shipdate': ['l_receiptdate', 'l_commitdate'],
    'l_receiptdate': ['l_shipdate', 'l_commitdate'],
    'l_commitdate': ['l_receiptdate', 'l_shipdate']
}
#correlations = {}
def table_sorting_options(table_name):
    single_table = extract_single_table(table_name)
    interesting_cols = extract_interesting_columns(single_table)
    pairs = itertools.product(interesting_cols, interesting_cols)
    
    total_times = []
    for pair in pairs:
        pruning_col = pair[0]
        sorted_col = pair[1]

        def compute_runtime(row):
            col_name = row['COLUMN_NAME']
            if pruning_col == sorted_col:
                if col_name == pruning_col:
                    return row['optimal_log_runtime']
                else:
                    if col_name in correlations.get(pruning_col, []):
                        # correlated to pruning column -> a lot of pruning, no sortedness
                        # TODO: better measure correlation
                        return 1.2 * row['optimal_runtime']
                    else:
                        return row['RUNTIME_NS']

            else:
                if col_name == pruning_col:
                    return row['optimal_runtime']
                elif col_name == sorted_col:
                    # TODO: should this be affected by correlation?
                    # we will get less chunks, so a linear scan should be close to optimal_runtime,
                    # but log time should beat it anyway
                    return row['log_runtime']
                else:
                    if col_name in correlations.get(pruning_col, []):
                        # correlated to pruning column -> a lot of pruning, no sortedness
                        # TODO: better measure correlation
                        return 1.2 * row['optimal_runtime']
                    else:
                        return row['RUNTIME_NS']

        effective_runtime = single_table.apply(compute_runtime, axis=1)
        total_times.append([pair, effective_runtime.sum()])    
    total_times = pd.DataFrame(total_times, columns=['columns', 'time'])    
    return total_times

options = table_sorting_options(TABLE)
options.sort_values(by=['time'], ascending=True)

In [ ]:
aggregates = pd.read_csv(f"{STATISTICS_PATH}/aggregates.csv", sep=',')

# it looks like column names are mixed up.
# COLUMN_NAME -> actually GROUP_BY_COLUMN_COUNT
# GROUP_BY_COLUMN_COUNT -> actually AGGREGATE_COLUMN_COUNT
# AGGREGATE_COLUMN_COUNT -> actually COLUMN_NAME

COL_NAME = 'AGGREGATE_COLUMN_COUNT'
GROUPBY_COL = 'COLUMN_NAME'
AGG_COL = 'GROUP_BY_COLUMN_COUNT'

# All aggregates have to read the entire table, so we cannot skip chunks.
# But getting all groups consecutive could provide a speedup
# As a result, we care only about aggregates with group by columns

interesting_aggregates = aggregates[aggregates[GROUPBY_COL] > 0]
stats = interesting_aggregates.groupby(['TABLE_NAME', COL_NAME])
out_columns = pd.DataFrame(stats['OUTPUT_ROWS'].max())
out_columns.sort_values(by=['TABLE_NAME', 'OUTPUT_ROWS'], ascending=[True, False])
aggregates[aggregates['COLUMN_TYPE'] == 'DATA']

In [ ]:
scan_time_per_column = scans.groupby(['COLUMN_NAME'])
accumulated_scan_times = pd.DataFrame(scan_time_per_column['RUNTIME_NS'].sum())
total_scan_runtime = accumulated_scan_times['RUNTIME_NS'].sum()
assert total_scan_runtime == scans['RUNTIME_NS'].sum(), f"{total_scan_runtime}, {scans['RUNTIME_NS'].sum()}"
print(f"total scan runtime: {total_scan_runtime}")

scan_time_per_column_prunable = scans[scans['useful_for_pruning']].groupby(['COLUMN_NAME'])
accumulated_prunable_scan_times = pd.DataFrame(scan_time_per_column_prunable['RUNTIME_NS'].sum())
total_prunable_scan_runtime = accumulated_prunable_scan_times['RUNTIME_NS'].sum()
print(f"total prunable scan runtime: {total_prunable_scan_runtime}")
print(f"{100*total_prunable_scan_runtime/total_scan_runtime}% of scan runtime amount to prunable scans")

accumulated_scan_times.sort_values(['RUNTIME_NS'], ascending=False)

In [ ]:
joins = load_join_statistics()

print(joins['PROBE_COLUMN'].unique())

join_time_per_column = joins.groupby(['PROBE_COLUMN'])

accumulated_join_times = pd.DataFrame(join_time_per_column['RUNTIME_NS'].sum())
print(len(accumulated_join_times))
total_join_runtime = accumulated_join_times['RUNTIME_NS'].sum()
#assert total_join_runtime == joins['RUNTIME_NS'].sum(), f"{total_join_runtime},{joins['RUNTIME_NS'].sum()}"
print(f"total join runtime: {total_join_runtime}")

joins[joins.apply(lambda x : x['PROBE_COLUMN'] not in ['o_custkey' ,'n_nationkey' ,'s_nationkey' ,'l_suppkey', 's_suppkey',
 'l_orderkey', 'o_orderkey', 'p_partkey' ,'l_partkey' ,'ps_suppkey',
 'c_nationkey' ,'r_regionkey' ,'c_custkey' ,'ps_partkey'] ,axis=1)]

print(f"for {BENCHMARK}, joins take about {total_join_runtime / total_scan_runtime} times longer than table scans")
accumulated_join_times.sort_values(['RUNTIME_NS'], ascending=False)